In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arcqa-logical/__results__.html
/kaggle/input/arcqa-logical/arcqa_logical_qa.csv
/kaggle/input/arcqa-logical/__notebook__.ipynb
/kaggle/input/arcqa-logical/__output__.json
/kaggle/input/arcqa-logical/custom.css


In [2]:
!pip install -q transformers accelerate bitsandbytes peft trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("hugging_face")

In [4]:
from huggingface_hub import login
login(new_session=False)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit for QLoRA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,              # 4-bit quantization
    device_map="auto",              # Automatically split layers to GPU
    bnb_4bit_quant_type="nf4",      # NormalFloat4 (better for QLoRA)
    bnb_4bit_use_double_quant=True, # More compression
    torch_dtype="auto"
)

# Prepare for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,            # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

print("Model ready for QLoRA fine-tuning!")


2025-08-11 06:49:06.975321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754894947.161705      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754894947.220829      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model ready for QLoRA fine-tuning!


In [6]:
import pandas as pd
df = pd.read_csv('/kaggle/input/arcqa-logical/arcqa_logical_qa.csv')

In [7]:
df.head()

,question,correct_combinations,incorrect_combinations,correct_answers,incorrect_answers
0,In the symbiotic relationship between green al...,"[""The green algae in lichens produce food thro...","[""The green algae produce food for the fungi, ...",food,protection
1,"How are rocks classified, and what factors inf...","['Rocks are classified as igneous, metamorphic...",['Rocks are classified by their color and shap...,how they formed,their color; their shape
2,Why is calcium carbonate commonly used in chew...,['Calcium carbonate neutralizes digestive acid...,['Calcium carbonate has a pleasant flavor and ...,neutralizes digestive acid.,has a pleasant flavor.
3,A group of students are researching changes in...,['Tree rings can indicate historical temperatu...,['Carbon dating is used to determine the age o...,tree rings,carbon dating
4,How do decomposers contribute to soil fertilit...,"['Decomposers add nitrogen to the soil, which ...",['Decomposers remove excess water from the soi...,by adding nitrogen,by removing excess water; by removing excess m...


In [8]:
import random
import ast
import pandas as pd
import numpy as np


def safe_eval(val):
    if isinstance(val, list):  # already list
        return val
    if pd.isna(val):  # NaN
        return []
    if isinstance(val, str):  # string that looks like a list
        return ast.literal_eval(val)
    return []  # fallback

df['correct_combinations'] = df['correct_combinations'].apply(safe_eval)
df['incorrect_combinations'] = df['incorrect_combinations'].apply(safe_eval)


qa_pairs = []
correct_label_cycle = ['A', 'B', 'C', 'D']
label_index = 0  # To cycle through correct answer positions


for idx, row in df.iterrows():
    if len(row['correct_combinations']) == 0 or len(row['incorrect_combinations']) == 0:
        continue  # skip if missing answers
    
    question = row['question']
    correct_ans = row['correct_combinations'][0]
    
    incorrect_list = row['incorrect_combinations']
    if len(incorrect_list) >= 3:
        incorrect_ans = random.sample(incorrect_list, 3)
    else:
        incorrect_ans = random.choices(incorrect_list, k=3)
    
    # We'll fix the correct answer position according to label_index
    labels = ['A', 'B', 'C', 'D']
    correct_label = correct_label_cycle[label_index % 4]
    label_index += 1
    
    # Place correct answer in the position indicated by correct_label
    options = incorrect_ans.copy()
    options.insert(labels.index(correct_label), correct_ans)
    
    qa_pairs.append({
        'question': question,
        'A': options[0],
        'B': options[1],
        'C': options[2],
        'D': options[3],
        'correct_label': correct_label,
        'correct_answer_text': correct_ans
    })

qa_df = pd.DataFrame(qa_pairs)
print(f"Generated {len(qa_df)} QA pairs with balanced correct answer positions.")

qa_df.to_csv('arcqa_pairs.csv',index=False)


Generated 441 QA pairs with balanced correct answer positions.


In [9]:
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text
0,In the symbiotic relationship between green al...,The green algae in lichens produce food throug...,The green algae create food for the fungi and ...,Although green algae are responsible for produ...,"The green algae produce food for the fungi, wh...",A,The green algae in lichens produce food throug...
1,"How are rocks classified, and what factors inf...",Rocks are formed through processes that affect...,"Rocks are classified as igneous, metamorphic, ...",Although rocks are classified based on their f...,"Rocks are classified by their color and shape,...",B,"Rocks are classified as igneous, metamorphic, ..."
2,Why is calcium carbonate commonly used in chew...,Calcium carbonate is a popular choice for chew...,Although calcium carbonate is effective in neu...,Calcium carbonate neutralizes digestive acid a...,Calcium carbonate has a pleasant flavor and is...,C,Calcium carbonate neutralizes digestive acid a...
3,A group of students are researching changes in...,Carbon dating is used to determine the age of ...,"While tree rings indicate growth patterns, car...",Tree rings provide evidence of temperature cha...,Tree rings can indicate historical temperature...,D,Tree rings can indicate historical temperature...
4,How do decomposers contribute to soil fertilit...,"Decomposers add nitrogen to the soil, which en...",While decomposers do prevent the buildup of de...,"Decomposers remove excess water from the soil,...",Decomposers increase fertility by removing exc...,A,"Decomposers add nitrogen to the soil, which en..."


In [10]:
def zero_shot_qa(model, tokenizer, qa_dataframe):
    """
    Performs zero-shot question answering on a DataFrame of multiple-choice questions,
    tokenizing the input correctly to avoid errors.

    Args:
        model: A language model compatible with the specified prompting format.
        tokenizer: The tokenizer for the language model.
        qa_dataframe: A pandas DataFrame with columns 'question', 'A', 'B', 'C', 'D',
                      and 'correct_label'.

    Returns:
        A list of dictionaries, where each dictionary contains the original question,
        the model's predicted label, and the correct label.
    """
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    # Define the set of valid answer choices for quick lookup
    valid_choices = {'A', 'B', 'C', 'D'}

    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:
            # Construct the prompt with a clear, strict instruction
            prompt = f"""Question:
{str(row['question'])}

Options:
A. {str(row['A'])}
B. {str(row['B'])}
C. {str(row['C'])}
D. {str(row['D'])}

Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
Answer:"""

            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=2,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            
            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper()
        })
            
    return results

    

# Usage example
results = zero_shot_qa(model,tokenizer, qa_df)
my_results = pd.DataFrame(results)
my_results.to_csv('arcqa_zeroshot_results.csv',index=False)

for res in results[:20]:
    print(f"Q: {res['question']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

Q: In the symbiotic relationship between green algae and fungi in lichens, what role does the green algae play in supporting the fungi?
Predicted answer: A
Correct answer: A
-----
Q: How are rocks classified, and what factors influence this classification?
Predicted answer: B
Correct answer: B
-----
Q: Why is calcium carbonate commonly used in chewable tablets for stomach discomfort?
Predicted answer: C
Correct answer: C
-----
Q: A group of students are researching changes in climate in their region. Which of these options would provide the best evidence of changes in temperature and rainfall over several years?
Predicted answer: D
Correct answer: D
-----
Q: How do decomposers contribute to soil fertility and the management of dead organic matter in the environment?
Predicted answer: A
Correct answer: A
-----
Q: What substance is formed when oxygen combines with hydrogen, and what are some of its properties?
Predicted answer: B
Correct answer: B
-----
Q: Which of the following statemen

In [11]:
my_results.head()

,question,predicted_label,correct_label
0,In the symbiotic relationship between green al...,A,A
1,"How are rocks classified, and what factors inf...",B,B
2,Why is calcium carbonate commonly used in chew...,C,C
3,A group of students are researching changes in...,D,D
4,How do decomposers contribute to soil fertilit...,A,A
